# Data extraction and selection of the subjects

In [1]:
# Version of python
import sys
print(sys.version)

3.9.9 (v3.9.9:ccb0e6a345, Nov 15 2021, 13:06:05) 
[Clang 13.0.0 (clang-1300.0.29.3)]


In [2]:
import pandas as pd
import time
import os
import glob
import datetime
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
print("pandas",pd.__version__)
print("numpy",np.__version__)
print("matplotlib",mpl.__version__)

### setup ###
# Data Directory
data_dir = '/Users/chidakatsuyuki/Desktop/Project/raw_JMDC/'
current_path = "/Users/chidakatsuyuki/Desktop/Project/Codes_for_NS_article/"
output_csv_dir = current_path + 'NephroticSyndrome_csv/'

id_colname = '加入者ID' # enrollee ID
select_key = 'ICD10小分類コード' # ICD10 code 3 digits
suspicious_flag = '疑いフラグ' # a suspicious flag for a uncertain diagnosis
# select columns of the csv files


# These columns are Japanese because our original data stored in Japanese. We noted some important columns in English below. 
# '加入者ID' means patinet ID.
# 'ICD10小分類コード' and 'ICD10小分類名' means 3-digits ICD10 code and its discription.
# '診療年月' means the date of the medical consultation.
# '疑いフラグ' is a suspicious flag for a uncertain diagnosis.
# '死亡フラグ' means that patients have passed away
# '診療開始年月日' is the first date when the disease was diagnosed
column_select = {
    '傷病':['加入者ID','レセID','明細ID','ICD10小分類コード','ICD10小分類名','診療年月','医療施設ID',
                         '標準傷病コード','標準病名','主傷病フラグ','入院の契機となった傷病フラグ',
                         '疑いフラグ','診療開始年月日','転帰','死亡フラグ','増悪フラグ'],
    'レセプト':['加入者ID','レセID','レセ種別','診療年月',
                         '医療施設ID','診療科大分類','診療科中分類','診療実日数','入院日',
                         '退院日','総点数',],
    '診療行為':['加入者ID','レセID','明細ID','実施年月日',
                         '診療点数早見表区分コード','標準化診療行為コード',
                         'ICD9CM_level1','ICD9CM_level2','ICD9CM_level3',
                         '回数','数量','データ規格','同時ID','診療区分',
                         '1回あたり点数','1回あたり加算割合','1回あたり金額','レセプト記載点数'],
    '医薬品':['加入者ID','レセID','明細ID','医薬品コード',
                         '薬価','処方日','調剤日','1処方あたりの1日投与量','投与量単位',
                         '1処方あたりの投与日数','投薬量','同時ID','頓服フラグ','診療区分',
                         '調剤料','薬剤料','加算料','レセプト記載点数'],
    '材料':['加入者ID','レセID','明細ID','使用年月日',
                         '診療点数早見表区分コード','標準化材料コード','レセプト記載単位','マスタ単位','1回あたりの使用量',
                         '回数','同時ID','診療区分',
                         '材料価格','材料加算割合','レセプト記載点数'],
}

pandas 1.3.4
numpy 1.22.4
matplotlib 3.6.3


In [3]:
def getTargetPatient(code_list: list = ["N04"], 
                     select_key: str = select_key,
                     id_colname: str = id_colname
                    )-> [list,list]:
    
    
    
    
    """
    getTargetPatient()
    This extracts candidates in the case and control group.
    code_list      : Taget codes used for the extraction
    select_key     : column name used for the extraction
    id_colname     : column name for ID
      return
         patient_parameter: list of patient candidate 
         control_parameter: list of control candidate 
     """
    file_name = '傷病.csv' # original data file of diagnosis information
    # We set　a chunksize
    paths = glob.glob
    reader = pd.read_csv(data_dir + file_name, encoding='cp932',
                usecols=[id_colname, select_key, suspicious_flag],
                chunksize = 1000000)

    df = pd.concat((r for r in reader), ignore_index=True)
    
    all_patient = df[id_colname].unique()
    print(datetime.datetime.now(),'Total number: unique patient IDs:',len(all_patient))

    target_code_patient = df.loc[df[select_key].isin(code_list), id_colname].unique()    
    print(datetime.datetime.now(),f'Unique IDs with the target disease:',len(target_code_patient))

    patient_parameter = df.loc[(df[select_key].isin(code_list))&(df[suspicious_flag]!=1), id_colname].unique()    
    print(datetime.datetime.now(),
          f'Unique IDs with the target disease excluding a suspicious flag i.e. case group candidates :',
          len(patient_parameter))

    control_parameter = list(set(all_patient) - set(target_code_patient))  
    print(datetime.datetime.now(),
          'Unique IDs without the target disease i.e. control group candidates:',
          len(control_parameter))

    return [patient_parameter,control_parameter]
    

    

def extractData(file_title: str, 
                use_col: bool, 
                id_list: list, 
                output_csv_path: str, 
                id_colname: str = id_colname)-> pd.DataFrame:

    
    
    
    """
    extractData()
    This extracts and output information of seleceted IDs from specified data
      input
         file_title     : original data file path
         use_col        : select columns or not (boolean value)
         id_list        : IDs
         id_colname     : column name for ID
         output_csv_path: output path for a csv file
         
      return
         df: Selected information dataframe 
    """
    df=pd.DataFrame()
    # Make path unless the path exists
    wk_dir = os.path.dirname(output_csv_path)
    if not os.path.exists(wk_dir):
        os.mkdir(wk_dir)

    paths = glob.glob(data_dir + file_title+'.csv')
    for path in paths:
        print(path)
        if use_col:
            reader = pd.read_csv(path, encoding='cp932',
                                 usecols = column_select.get(file_title),
                                 chunksize = 1000000)
        else:        
            reader = pd.read_csv(path, encoding='cp932',
                                 chunksize = 1000000)
        # Concatenate with keeping same IDs
        df = pd.concat((r[r[id_colname].isin(id_list)] for r in reader), ignore_index=True)

    #　 Output csv file
    df.to_csv(output_csv_path, index=False)

    print(datetime.datetime.now(),':output CSV:',output_csv_path)

    return df
    

In [4]:
def getCodeslist(file_path:str)->list:
    
    
    
    """
    getCodeslist(file_path)
     This return ICD10 code list and disease code list from a tsv file.
      input
         file_path : path of the tsv file.
      return
        code_list: code list
    """
    # Read a tsv file.
    df = pd.read_csv(file_path, header=None, sep="\t")
    # The second column is disease list
    code_list = df[1]
    print(len(code_list))
    return code_list

In [5]:
patient_parameter, control_parameter = getTargetPatient()

2024-05-17 18:37:40.010343 Total number: unique patient IDs: 329500
2024-05-17 18:37:42.551103 Unique IDs with the target disease: 2057
2024-05-17 18:37:44.671700 Unique IDs with the target disease excluding a suspicious flag i.e. case group candidates : 613
2024-05-17 18:37:44.819207 Unique IDs without the target disease i.e. control group candidates: 327443


In [6]:
# select key & code
select_key = select_key
code_list = ['N04']

patient_parameter=[] 
control_parameter=[] 

##### main #####
# Time
start = time.time()
print(datetime.datetime.now(),':start')

# Extract candidate IDs
patient_parameter, control_parameter = getTargetPatient()

# Output csv of candidate IDs for both group
df_inj_p_param = extractData('傷病', True, patient_parameter, output_csv_dir+'Disease_of_case_candidates.csv')
df_patient_p_param = extractData('患者', False, patient_parameter, output_csv_dir+'Personal_of_case_candidate.csv')

end = time.time()
print (datetime.datetime.now(),':elapsed_time:{0}'.format(end-start) + "[sec]")


2024-05-17 18:37:45.402592 :start
2024-05-17 18:43:12.366135 Total number: unique patient IDs: 329500
2024-05-17 18:43:16.462333 Unique IDs with the target disease: 2057
2024-05-17 18:43:19.502289 Unique IDs with the target disease excluding a suspicious flag i.e. case group candidates : 613
2024-05-17 18:43:19.676662 Unique IDs without the target disease i.e. control group candidates: 327443
/Users/chidakatsuyuki/Desktop/Project/raw_JMDC/傷病.csv
2024-05-17 18:50:52.983490 :output CSV: /Users/chidakatsuyuki/Desktop/Project/Codes_for_NS_article/NephroticSyndrome_csv/Disease_of_case_candidates.csv
/Users/chidakatsuyuki/Desktop/Project/raw_JMDC/患者.csv
2024-05-17 18:50:54.431705 :output CSV: /Users/chidakatsuyuki/Desktop/Project/Codes_for_NS_article/NephroticSyndrome_csv/Personal_of_case_candidate.csv
2024-05-17 18:50:54.432722 :elapsed_time:789.0299932956696[sec]


# Set the case group through excluding IDs diagnosed as NS before their observation starts from candidates.
 1. Add additional information
     - Diagnosis_date (for the target disease)
     - Time (months)from the observation start to the diagnosis date
     - Age at the observation start

In [33]:
id_colname = '加入者ID' # enrollee ID
select_key = 'ICD10小分類コード' # ICD10 code 3 digits
suspicious_flag = '疑いフラグ'
obs_start_yyyymm = '観察開始年月' # the date of the obseavation start 
obs_end_yyyymm = '観察終了年月' # the date of the obseavation end
code_list = ['N04']
birth_yyyymm = '加入者生年月' # birth date of individuals
first_diag_yyyymmdd = '診療開始年月日' # the first diagnosis date of each disease

target_diag_yyyymm = 'Target_diag_yyyymm' # the first diagnosis date of target disease
observation_peiod= "Obsevation_length" 
diag_from_obs = 'Taget_diag_from_start' # Time (months) from the observation start to the diagnosis date
age_obs_start = 'Age_at_start' # age at the observation start




# Personal information of case candidates
df_pp = pd.read_csv(output_csv_dir+'Personal_of_case_candidate.csv')

# Set the diagnosis date same as the earliest first date when the disease was diagnosed
# If you used several codes for the first selection, the first code among them will be used.
df_inj = pd.read_csv(output_csv_dir+'Disease_of_case_candidates.csv')
"""ADD"""
df_inj = df_inj[df_inj[suspicious_flag]!=1]
df_inj = (df_inj[df_inj[select_key].isin(code_list)].groupby(id_colname).min()
          [[first_diag_yyyymmdd]].reset_index()
          .rename(columns={first_diag_yyyymmdd:target_diag_yyyymm})
          .astype({target_diag_yyyymm:int})
         )

# Concatenate diagnosis records with personal information
df_patient_param = pd.merge(df_pp, df_inj, on=id_colname, how='left')

# Convet target diagnosis date units into months units
df_patient_param[target_diag_yyyymm] = pd.to_datetime(df_patient_param[target_diag_yyyymm],format='%Y%m%d').dt.strftime('%Y%m')

# Make a column of time (months) from the observation start to the diagnosis date
df_patient_param[diag_from_obs] = (
           pd.to_datetime(df_patient_param[target_diag_yyyymm], format='%Y%m').dt.to_period('M').view('int')
         - pd.to_datetime(df_patient_param[obs_start_yyyymm], format='%Y%m').dt.to_period('M').view('int')
           )
# Make a column of ages at the observation start 
df_patient_param[age_obs_start] = (
             pd.to_datetime(df_patient_param[obs_start_yyyymm],format='%Y%m').dt.to_period('M').view('int')
           - pd.to_datetime(df_patient_param[birth_yyyymm],format='%Y%m').dt.to_period('M').view('int') ) // 12  

# The month in which the observation start set 0
df_patient_param[observation_peiod]  = (
             pd.to_datetime(df_patient_param[obs_end_yyyymm],format='%Y%m').dt.to_period('M').view('int')
           - pd.to_datetime(df_patient_param[obs_start_yyyymm],format='%Y%m').dt.to_period('M').view('int'))

    
# Exclude IDs diagnosed as NS before their observation starts (case group)
df_patient = df_patient_param[df_patient_param[diag_from_obs]>=0]
patient_list = df_patient[id_colname].unique()
# Actually this is not list but numpy.ndarray

print('Number_of_the_case :',len(patient_list))

df_patient.head(2)

Number_of_the_case : 411


,加入者ID,加入者生年月,加入者性別,本人家族,観察開始年月,観察終了年月,観察終了理由(死亡)フラグ,Target_diag_yyyymm,Taget_diag_from_start,Age_at_start,Obsevation_length
1,M000051963,197411,女性,家族,201101,201903,NaN,201411,46,36,98
2,M000055609,195203,男性,本人,201203,201903,NaN,201504,37,60,84


# Extract control cadidates information
1. Add additional information
     - Age at the observation start

In [34]:
id_colname = '加入者ID' # enrollee ID
file_name = '患者.csv' # original file for personal data

# All personal data used for matching
df_control_param = pd.read_csv(data_dir + file_name, encoding='cp932')

# IDs whitout N04 diagnosis
df_control_param = df_control_param.loc[df_control_param[id_colname].isin(control_parameter)]

# Make a column of ages at the observation start 
df_control_param.loc[:,age_obs_start] = (
             pd.to_datetime(df_control_param[obs_start_yyyymm],format='%Y%m').dt.to_period('M').view('int')
           - pd.to_datetime(df_control_param[birth_yyyymm],format='%Y%m').dt.to_period('M').view('int') ) // 12   

# The month in which the observation start set 0
df_control_param.loc[:,observation_peiod]  = (
             pd.to_datetime(df_control_param[obs_end_yyyymm],format='%Y%m').dt.to_period('M').view('int')
           - pd.to_datetime(df_control_param[obs_start_yyyymm],format='%Y%m').dt.to_period('M').view('int'))

print('Number of the case candidates:', df_control_param[id_colname].nunique())

df_control_param.head(2)

Number of the case candidates: 327443


,加入者ID,加入者生年月,加入者性別,本人家族,観察開始年月,観察終了年月,観察終了理由(死亡)フラグ,Age_at_start,Obsevation_length
0,M000000133,195606,男性,本人,201201,201903,NaN,55,86
1,M000000141,198209,男性,本人,200801,201903,NaN,25,134


# Set the control group through matching 1:1 to the case group 

1. Criteria
     - Same sex
     - The nearest age at the observation start (eventually same age) 
     - The nearest observation periods
2. Add additional information
     - Matched_flag on the control IDs to avoid matching same ID again
     - Matched_IDs on the case IDs to connect matched control IDs

In [35]:
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from dateutil.relativedelta import relativedelta

id_colname = '加入者ID' # enrollee ID
select_key = 'ICD10小分類コード' # ICD10 code 3 digits
sex_colname = '加入者性別'
age_obs_start = 'Age_at_start'


def get_matching_pairs(treated_df:pd.DataFrame,
                       non_treated_df: pd.DataFrame , 
                       scaler:bool =True, 
                       n_neighbors: int =1
                      )-> pd.DataFrame:
    
    """
    get_matching_pairs()
     input
      treated_df: the case group with information used for macthing
      non_treated_df: the control candidates with information used for macthing
      scaler: need for scaling to use the nearest neighbor method
      n_neighbors: number of neighbors i.e. matched control candidates
     return
      matched: dataframe of the matched group
    """

    treated_x = treated_df.values
    non_treated_x = non_treated_df.values

    if scaler:
        # Standardization　based on the case group
        scaler = StandardScaler()
        scaler.fit(treated_x)
        treated_x = scaler.transform(treated_x)
        non_treated_x = scaler.transform(non_treated_x)

    nbrs = NearestNeighbors(n_neighbors=n_neighbors, algorithm='ball_tree').fit(non_treated_x)
    distances, indices = nbrs.kneighbors(treated_x)

    if n_neighbors==1:
        indices = indices[:,0]
        indices = indices.reshape(indices.shape[0])
    elif n_neighbors==2:
        indices = np.concatenate([indices[:,0],indices[:,1]])
        
    matched = non_treated_df.iloc[indices]
    
    return matched


df_control_param['matched_flg'] = 0
df_patient['matched_ID'] = ""

# Set enrollee ID as dataframe index
df_patient = df_patient.set_index(id_colname)
df_control_param = df_control_param.set_index(id_colname)

for patient in df_patient.index:
    # Get sex and age individually
    p_sex = df_patient.at[patient,sex_colname]
    p_age = df_patient.at[patient,age_obs_start]
    p_duaring = df_patient.at[patient,observation_peiod]
    p_date = df_patient.at[patient, diag_from_obs]

    # Explorable control candidates that have the same sex without matched_flag
    df_match = df_control_param[(df_control_param[sex_colname]==p_sex)
                                &(df_control_param[age_obs_start]==p_age)
                                &(df_control_param.matched_flg==0)].copy()

    # Patient dataframe for matching
    treated_df = pd.DataFrame({age_obs_start:[p_age],
                               observation_peiod:[p_duaring]})
    # Control dataframe for matching
    non_treated_df = df_match[[age_obs_start,observation_peiod]]
    
    # Get the nearest candidates index, and then IDs
    df_matched = get_matching_pairs(treated_df, non_treated_df, True, 1)
    matched_id = df_matched.index[0]

    # Add matched flags
    df_control_param.at[matched_id, 'matched_flg']=1
    df_patient.at[patient, 'matched_ID']=matched_id
    # Assume from control's observation start to their target diagnosis as the same period of the matched case's one
    df_control_param.at[matched_id, target_diag_yyyymm]=(relativedelta(months=p_date)+
                                                           pd.to_datetime(df_control_param.at[matched_id,obs_start_yyyymm],format="%Y%m")).strftime("%Y%m")
# Return to normal indices
df_patient.reset_index(inplace=True)
df_control_param.reset_index(inplace=True)

# Collect matched control IDs
df_control_param = df_control_param[df_control_param.matched_flg==1]
control_list = df_control_param[id_colname].unique()
# Actually this is not list but numpy.ndarray

print('matched_control ID:',len(control_list))

# Output the case and control group as csv files
# Case
wk_dir = output_csv_dir+'patient/'
if not os.path.exists(wk_dir):
    os.mkdir(wk_dir)
df_patient.to_csv(wk_dir +'indnividuals.csv',index=False)

# Control
wk_dir = output_csv_dir+'control/'
if not os.path.exists(wk_dir):
    os.mkdir(wk_dir)
df_control_param.to_csv(wk_dir +'individuals.csv',index=False)

df_control_param.head(2)

/var/folders/y4/t8nz0k_j4nscdlgrd210j50c0000gn/T/ipykernel_64412/1956466701.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_patient['matched_ID'] = ""


matched_control ID: 411


,加入者ID,加入者生年月,加入者性別,本人家族,観察開始年月,観察終了年月,観察終了理由(死亡)フラグ,Age_at_start,Obsevation_length,matched_flg,Target_diag_yyyymm
6,M000000672,197906,男性,本人,200903,201803,NaN,29,108,1,201301
9,M000000706,196207,女性,家族,201304,201903,NaN,50,71,1,201812


In [36]:
# Count case IDs of which ages are same as ones of control IDs 
df_pat = df_patient
df_cont = df_control_param
tmp = pd.merge(df_pat[['matched_ID',age_obs_start]],df_cont[[id_colname,age_obs_start]], left_on='matched_ID',right_on=id_colname)
print(df_pat.matched_ID.count())
print(tmp[tmp[age_obs_start+"_x"]==tmp[age_obs_start+"_y"]].matched_ID.count())
# Individuals in both group have their ages same to the matched opponents.

411
411


# Output csv files for further analysis.

In [37]:
# Group information
dict_pat = {id_colname:patient_list.tolist(),"Target":np.ones(len(patient_list)).tolist()}
dict_cont = {id_colname:control_list.tolist(),"Target":np.zeros(len(control_list)).tolist()}
df_groups=pd.concat([pd.DataFrame(dict_pat),pd.DataFrame(dict_cont)])
df_groups.to_csv(output_csv_dir+"group.csv",index=False)

In [38]:
id_colname = '加入者ID' # enrollee ID
code_colname = 'ICD10小分類コード'
code_desc = 'ICD10小分類名' # 
suspicious_flag = '疑いフラグ'
death_flag = '死亡フラグ'
obs_start_yyyymm = '観察開始年月' # the date of the obseavation start 
obs_end_yyyymm = '観察終了年月' # the date of the obseavation end
code_list = ['N04']
birth_yyyymm = '加入者生年月' # birth date of individuals
records_date = '診療年月' # the diagnosis date of the record
first_diag_yyyymmdd = '診療開始年月日' # the first diagnosis date of each disease

In [39]:
# Select diagnosis in the group from original data 
column_select['傷病']=[id_colname,code_colname,records_date,first_diag_yyyymmdd, death_flag, suspicious_flag]
df_inj_p = extractData('傷病', True, patient_list, output_csv_dir+'patient/Group_diseases_0.csv')
df_inj_c = extractData('傷病', True, control_list, output_csv_dir+'control/Group_diseases_0.csv')

/Users/chidakatsuyuki/Desktop/Project/raw_JMDC/傷病.csv
2024-05-28 06:10:18.254160 :output CSV: /Users/chidakatsuyuki/Desktop/Project/Codes_for_NS_article/NephroticSyndrome_csv/patient/Group_diseases_0.csv
/Users/chidakatsuyuki/Desktop/Project/raw_JMDC/傷病.csv
2024-05-28 06:19:14.883043 :output CSV: /Users/chidakatsuyuki/Desktop/Project/Codes_for_NS_article/NephroticSyndrome_csv/control/Group_diseases_0.csv


In [40]:
# Exclude a suspicious diagnosis based on the flag==1
df_inj_p = df_inj_p[df_inj_p[suspicious_flag]!=1]
df_inj_c = df_inj_c[df_inj_c[suspicious_flag]!=1]

# Join personal information
df_p = pd.merge(df_inj_p, df_pat, on=id_colname, how='left')
df_p = df_p.sort_values(id_colname)

df_c = pd.merge(df_inj_c, df_cont, on=id_colname, how='left')
df_c = df_c.sort_values(id_colname)

# Record dates based on the target diagnosis date
df_p["Months_from_target_diag"]  = (
             pd.to_datetime(df_p[records_date],format='%Y%m').dt.to_period('M').view('int')
           - pd.to_datetime(df_p[target_diag_yyyymm],format='%Y%m').dt.to_period('M').view('int'))

df_c["Months_from_target_diag"] = ( 
            pd.to_datetime(df_c[records_date],format='%Y%m').dt.to_period('M').view('int')
            - pd.to_datetime(df_c[target_diag_yyyymm],format='%Y%m').dt.to_period('M').view('int'))

In [41]:
# Target label
df_p["Target"]=1
df_c["Target"]=0

In [42]:
# Column names into English
english_trans_dict={
    '加入者ID':"Enrollee ID",
    '疑いフラグ':"Suspicious_flag",
    '死亡フラグ':"Death_flag",
    '診療年月': "Record_yyyymm",
    'ICD10小分類コード': "ICD10_3_Code",
    "加入者性別": "Sex",
    "加入者生年月": "Birth_date",
    "本人家族":"Inssured_or_Family",
    "観察終了理由(死亡)フラグ":"Death_censor",
    '観察開始年月':"Observation_start_yyyymm",
    '観察終了年月':"Observation_end_yyyymm",
    '診療開始年月日':"First_diagnosis_yyyymmdd" 
}

# Translate Japanese categorical values into English ones
english_val_trans_dict= {
    "女性":"Female","男性":"Male",
    "家族":"Family","本人":"Inssured_person"
}

for df in [df_p,df_c,df_pat,df_cont]:
    df.rename(columns=english_trans_dict, inplace=True)
    df.replace(english_val_trans_dict, inplace=True)


In [43]:
# Output csv
df_p.to_csv(output_csv_dir+'patient/Group_diseases.csv',index=False)
df_c.to_csv(output_csv_dir+'control/Group_diseases.csv',index=False)

In [44]:
# Case
wk_dir = output_csv_dir+'patient/' 
df_patient.to_csv(wk_dir +'individuals.csv',index=False)

# Control
wk_dir = output_csv_dir+'control/' 
df_control_param.to_csv(wk_dir +'individuals.csv',index=False)


In [45]:
# Other files
_ = extractData('患者', False, df_groups[id_colname], output_csv_dir+'Personals.csv')
_ = extractData('健診', False, df_groups[id_colname], output_csv_dir+'Checkups.csv')
_ = extractData('傷病', False, df_groups[id_colname], output_csv_dir+'Diseases.csv')

/Users/chidakatsuyuki/Desktop/Project/raw_JMDC/患者.csv
2024-05-28 06:19:24.041019 :output CSV: /Users/chidakatsuyuki/Desktop/Project/Codes_for_NS_article/NephroticSyndrome_csv/Personals.csv
/Users/chidakatsuyuki/Desktop/Project/raw_JMDC/健診.csv
2024-05-28 06:19:55.793211 :output CSV: /Users/chidakatsuyuki/Desktop/Project/Codes_for_NS_article/NephroticSyndrome_csv/Checkups.csv
/Users/chidakatsuyuki/Desktop/Project/raw_JMDC/傷病.csv
2024-05-28 06:35:45.369570 :output CSV: /Users/chidakatsuyuki/Desktop/Project/Codes_for_NS_article/NephroticSyndrome_csv/Diseases.csv


In [46]:
# This function is for drug data of which encoding is different
def extractDataD(file_title: str, 
                use_col: bool, 
                id_list: list, 
                output_csv_path: str, 
                id_colname: str = id_colname)-> pd.DataFrame:

    
    
    
    """
    extractData()
    This extracts and output information of seleceted IDs from specified data
      input
         file_title     : original data file path
         use_col        : select columns or not (boolean value)
         id_list        : IDs
         id_colname     : column name for ID
         output_csv_path: output path for a csv file
         
      return
         df: Selected information dataframe 
    """
    df=pd.DataFrame()
    df_list=[]
    # Make path unless the path exists
    wk_dir = os.path.dirname(output_csv_path)
    if not os.path.exists(wk_dir):
        os.mkdir(wk_dir)

    paths = glob.glob(data_dir + file_title+'.csv')
    
    
    for path in paths:
        print(path)
        if use_col:
            reader = pd.read_csv(path,
                                 usecols = column_select.get(file_title),
                                 chunksize = 1000000)
        else:        
            reader = pd.read_csv(path,
                                 chunksize = 1000000)
        # Concatenate with keeping same IDs
        df_list.append(pd.concat((r[r[id_colname].isin(id_list)] for r in reader), ignore_index=True))
        df = pd.concat(df_list, ignore_index=True)
    #　 Output csv file
    df.to_csv(output_csv_path, index=False)

    print(datetime.datetime.now(),':output CSV:',output_csv_path)

    return df

In [47]:
# For drug data csv (utf-8), We chdanged encoding while reading csv
id_colname= "加入者ID"
df_drug = extractDataD('医薬品12_*', 
                       False, 
                      df_groups[id_colname], 
                      output_csv_dir+'Drugs.csv')

/Users/chidakatsuyuki/Desktop/Project/raw_JMDC/医薬品12_09.csv
/Users/chidakatsuyuki/Desktop/Project/raw_JMDC/医薬品12_08.csv
/Users/chidakatsuyuki/Desktop/Project/raw_JMDC/医薬品12_11.csv
/Users/chidakatsuyuki/Desktop/Project/raw_JMDC/医薬品12_05.csv
/Users/chidakatsuyuki/Desktop/Project/raw_JMDC/医薬品12_04.csv
/Users/chidakatsuyuki/Desktop/Project/raw_JMDC/医薬品12_10.csv
/Users/chidakatsuyuki/Desktop/Project/raw_JMDC/医薬品12_06.csv
/Users/chidakatsuyuki/Desktop/Project/raw_JMDC/医薬品12_12.csv
/Users/chidakatsuyuki/Desktop/Project/raw_JMDC/医薬品12_07.csv
/Users/chidakatsuyuki/Desktop/Project/raw_JMDC/医薬品12_03.csv
/Users/chidakatsuyuki/Desktop/Project/raw_JMDC/医薬品12_02.csv
/Users/chidakatsuyuki/Desktop/Project/raw_JMDC/医薬品12_01.csv
2024-05-28 07:02:59.419409 :output CSV: /Users/chidakatsuyuki/Desktop/Project/Codes_for_NS_article/NephroticSyndrome_csv/Drugs.csv
